Importing the Dependencies

In [3]:
!pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

In [2]:
diabetes_dataset = pd.read_csv('diabetes.csv')

In [3]:
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
diabetes_dataset.shape

(768, 9)

In [5]:
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
diabetes_dataset['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

0 --> Non-Diabetic

1 --> Diabetic

In [7]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [8]:
X = diabetes_dataset.drop(columns='Outcome', axis=1)
y = diabetes_dataset['Outcome']

In [9]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [10]:
for column in X.columns:
  print(column)

Pregnancies
Glucose
BloodPressure
SkinThickness
Insulin
BMI
DiabetesPedigreeFunction
Age


In [11]:
print(y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


Train Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [13]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


Training the Model

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [16]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)
model.save('diabetes_lstm_model.h5')
#model.save('diabetes_lstm_model.keras')

Epoch 1/50


18/18 [==============================] - 3s 45ms/step - loss: 0.6613 - accuracy: 0.6902 - val_loss: 0.6397 - val_accuracy: 0.7097
Epoch 2/50
18/18 [==============================] - 0s 6ms/step - loss: 0.6174 - accuracy: 0.7029 - val_loss: 0.5922 - val_accuracy: 0.7258
Epoch 3/50
18/18 [==============================] - 0s 6ms/step - loss: 0.5720 - accuracy: 0.7337 - val_loss: 0.5464 - val_accuracy: 0.7581
Epoch 4/50
18/18 [==============================] - 0s 7ms/step - loss: 0.5254 - accuracy: 0.7554 - val_loss: 0.5132 - val_accuracy: 0.7742
Epoch 5/50
18/18 [==============================] - 0s 7ms/step - loss: 0.4971 - accuracy: 0.7681 - val_loss: 0.4943 - val_accuracy: 0.7742
Epoch 6/50
18/18 [==============================] - 0s 7ms/step - loss: 0.4848 - accuracy: 0.7681 - val_loss: 0.4781 - val_accuracy: 0.7581
Epoch 7/50
18/18 [==============================] - 0s 6ms/step - loss: 0.4786 - accuracy: 0.7790 - val_loss: 0.4694 - val_accuracy: 0.7581
Epoch 8/50
18/18 

C:\Users\KAJAL\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
y_test_probabilities = model.predict(X_test_reshaped)
y_test_pred = (y_test_probabilities > 0.5).astype('int32')

5/5 [==============================] - 0s 3ms/step


Model Evaluation

Accuracy Score

In [20]:
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test Accuracy: ', test_accuracy)
print(classification_report(y_test, y_test_pred))


Test Accuracy:  0.7402597402597403
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       100
           1       0.65      0.56      0.60        54

    accuracy                           0.74       154
   macro avg       0.71      0.70      0.70       154
weighted avg       0.73      0.74      0.73       154



In [21]:
#(5,166,72,19,175,25.8,0.587,51)
sample_input = np.array([[1,85,66,23,94,28.1,0.167,21]])
sample_input_scaled = scaler.transform(sample_input)
sample_input_reshaped = sample_input_scaled.reshape((sample_input_scaled.shape[0], 1, sample_input_scaled.shape[1]))
prediction = model.predict(sample_input_reshaped)
# Display the prediction result
if prediction[0][0] > 0.5:
    print('The person is diabetic')
else:
    print('The person is not diabetic')

1/1 [==============================] - 0s 20ms/step
The person is not diabetic


C:\Users\KAJAL\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

diabetes_dataset = pd.read_csv('diabetes.csv')

X = diabetes_dataset.drop(columns='Outcome', axis=1)
y = diabetes_dataset['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifier = SVC(kernel='rbf', probability=True)

classifier.fit(X_train_scaled, y_train)

y_pred = classifier.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

import pickle
file = 'diabetes_model.h5'
pickle.dump(classifier, open(file, 'wb'))
model = pickle.load(open('diabetes_model.h5', 'rb'))

sample_input = np.array([[5, 166, 72, 19, 175, 25.8, 0.587, 51]])
sample_input_scaled = scaler.transform(sample_input)

prediction = model.predict(sample_input_scaled)
probability = model.predict_proba(sample_input_scaled)

print(f"Probability of being not diabetic:{probability[0][0]*100:.4f}%")
print(f"Probability of being diabetic:{probability[0][1]*100:.4f}%")

if prediction[0] == 1:
    print('Result:The person is diabetic')
else:
    print('Result:The person is not diabetic')

Model Accuracy: 0.7532
Probability of being not diabetic:26.4666%
Probability of being diabetic:73.5334%
Result:The person is diabetic


C:\Users\KAJAL\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
